In [1]:
import logging
import os
import random
import numpy as np
import pandas as pd

from src.configuration_space.LKH import CONFIGURATION_SPACE
from src.constant import DATA_DIR, MAX_WORKERS, TEMP_DIR
from src.experiment.GlobalExperiment import GlobalExperiment
from src.experiment.ParhydraExperiment import ParhydraExperiment
from src.experiment.PcitExperiment import PcitExperiment, _Clustering
from src.instance.TSP_Instance import TSP_Instance, TSP_InstanceSet
from src.portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.experiment.CepsExperiment import CepsExperiment
pd.options.display.max_columns = 999

In [2]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=30,
    seed=0,
)

In [20]:
train_instances, test_instances = TSP_InstanceSet.train_test_from_index_file(
    filepath=DATA_DIR / "TSP" / "CEPS_benchmark" / "index.json",
    train_size=30,
    seed=0,
)

t_c = int(1.5 * 3600)
t_v = int(0.5 * 3600)
K = 4
n = 10
t_ini = sum([i * train_instances.size * 10 for i in range(1, K + 1)]) * 10
t_i = 1200
max_iter = 4

In [ ]:
# t_c = 30
# t_v = 30
# K = 2
# n = 2
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = GlobalExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# t_c = 30
# t_v = 30
# K = 2
# n = 3
# solver_class = TSP_LKH_Solver
# instance_class= TSP_Instance

# experiment = ParhydraExperiment(
#     t_c=t_c,
#     t_v=t_v,
#     K=K,
#     n=n,
#     solver_class=solver_class,
#     instance_class=instance_class,
# )

# best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
t_c = 50
t_v = 100
K = 2
n = 2
max_iter = 2
solver_class = TSP_LKH_Solver
instance_class = TSP_Instance

experiment = PcitExperiment(
    t_c=t_c,
    t_v=t_v,
    K=K,
    n=n,
    max_iter=max_iter,
    solver_class=solver_class,
    instance_class=instance_class,
)

best_portfolio = experiment.construct_portfolio(train_instances)

In [ ]:
# remaining_time = np.ones(shape=(K, )) * np.inf
# best_portfolio.evaluate(test_instances, remaining_time, "test")

In [ ]:
from src.database import db_connect
conn = db_connect()

In [4]:
import sqlite3
# conn = sqlite3.connect("_archive/3/PARHYDRA.db")
conn = sqlite3.connect("database/2024_11_18_00_06_38.db")

In [5]:
df_solvers = pd.read_sql_query("SELECT * FROM solvers", conn)
df_solvers

,id,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,GAIN23,GAIN_CRITERION,INITIAL_STEP_SIZE,INITIAL_TOUR_ALGORITHM,INITIAL_TOUR_FRACTION,KICKS,KICK_TYPE,MAX_BREADTH,MAX_CANDIDATES,MOVE_TYPE,PATCHING_A,PATCHING_C,POPULATION_SIZE,RESTRICTED_SEARCH,SUBGRADIENT,SUBSEQUENT_MOVE_TYPE,SUBSEQUENT_PATCHING
0,2220588739511820280,52,1,YES,ALPHA,7,QUADRANT,NO,YES,1,QUICK-BORUVKA,0.582020,3,0,954863081,10,3,3,1,8,YES,YES,6,NO
1,2051815471081033189,45,0,YES,DELAUNAY,10,QUADRANT,YES,YES,3,SIERPINSKI,0.842342,3,5,1010150627,9,6,3,1,26,YES,NO,4,NO
2,574052600841816340,58,1,YES,QUADRANT,6,QUADRANT,NO,YES,5,GREEDY,0.569101,2,5,737143853,6,2,3,4,37,NO,NO,6,YES
3,1366896661489672079,47,0,YES,NEAREST-NEIGHBOR,4,QUADRANT,NO,NO,3,WALK,0.059646,1,5,1344107173,9,3,0,1,53,NO,YES,2,YES
4,1254857386554662322,60,0,NO,ALPHA,6,QUADRANT,YES,NO,1,QUICK-BORUVKA,0.710241,4,0,49135930,5,2,1,0,27,NO,NO,6,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,1461877696262894647,51,1,NO,QUADRANT,3,QUADRANT,NO,NO,1,WALK,0.498639,0,5,1638188432,10,3,1,2,2,NO,NO,4,YES
81,1314436244633729951,41,0,YES,NEAREST-NEIGHBOR,0,QUADRANT,YES,NO,5,WALK,0.091736,3,5,1885901726,2,5,5,0,55,YES,NO,4,YES
82,2270204218437147001,54,1,NO,DELAUNAY,2,QUADRANT,NO,YES,5,GREEDY,0.364680,4,4,175643380,10,5,0,1,20,YES,YES,3,YES
83,2300580964560483949,42,1,NO,NEAREST-NEIGHBOR,5,QUADRANT,YES,NO,2,QUICK-BORUVKA,0.569677,3,0,1246513937,7,5,4,3,61,YES,YES,0,NO


In [6]:
df_instances = pd.read_sql_query("SELECT * FROM instances", conn)
df_instances

,id
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp
1,data/TSP/CEPS_benchmark/grid/04.tsp
2,data/TSP/CEPS_benchmark/grid/33.tsp
3,data/TSP/CEPS_benchmark/rotation/45.tsp
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp
...,...
740,data/TSP/CEPS_benchmark/implosion/23.tsp
741,data/TSP/CEPS_benchmark/expansion/42.tsp
742,data/TSP/CEPS_benchmark/compression/17.tsp
743,data/TSP/CEPS_benchmark/cluster/47.tsp


In [7]:
df = pd.read_sql_query("SELECT * FROM results", conn)
# df.to_excel("tmp.xlsx")
df

,instance_id,solver_id,cost,time,comment,created_at
0,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,2220588739511820280,100.00,10.00,initialization,2024-11-17 23:06:49
1,data/TSP/CEPS_benchmark/grid/04.tsp,2220588739511820280,100.00,10.00,initialization,2024-11-17 23:06:50
2,data/TSP/CEPS_benchmark/grid/33.tsp,2220588739511820280,100.00,10.00,initialization,2024-11-17 23:06:50
3,data/TSP/CEPS_benchmark/rotation/45.tsp,2220588739511820280,100.00,10.00,initialization,2024-11-17 23:06:50
4,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp,2220588739511820280,100.00,10.00,initialization,2024-11-17 23:06:50
...,...,...,...,...,...,...
8898,data/TSP/CEPS_benchmark/cluster/47.tsp,1482672950880257277,4.55,4.55,test,2024-11-18 06:47:17
8899,data/TSP/CEPS_benchmark/expansion/22.tsp,1317975573233644005,100.00,10.00,test,2024-11-18 06:47:18
8900,data/TSP/CEPS_benchmark/expansion/22.tsp,721382164073135269,100.00,10.00,test,2024-11-18 06:47:23
8901,data/TSP/CEPS_benchmark/expansion/22.tsp,2300580964560483949,100.00,10.00,test,2024-11-18 06:47:23


In [8]:
df["comment"].value_counts()

configuration     4925
test              1880
validation        1049
mutation           948
initialization      60
pre_mutation        41
Name: comment, dtype: int64

In [9]:
df.loc[df["comment"] == "configuration"]

,instance_id,solver_id,cost,time,comment,created_at
60,data/TSP/CEPS_benchmark/cluster_netgen/40.tsp,574052600841816340,100.0,10.0,configuration,2024-11-17 23:08:54
61,data/TSP/CEPS_benchmark/grid/04.tsp,574052600841816340,100.0,10.0,configuration,2024-11-17 23:08:56
62,data/TSP/CEPS_benchmark/grid/33.tsp,574052600841816340,100.0,10.0,configuration,2024-11-17 23:08:58
63,data/TSP/CEPS_benchmark/rotation/45.tsp,574052600841816340,100.0,10.0,configuration,2024-11-17 23:08:58
64,data/TSP/CEPS_benchmark/uniform_portgen/11.tsp,574052600841816340,100.0,10.0,configuration,2024-11-17 23:08:58
...,...,...,...,...,...,...
6958,data/TSP/CEPS_generated/86.tsp,1645744692394970611,100.0,10.0,configuration,2024-11-18 06:12:09
6959,data/TSP/CEPS_generated/87.tsp,1645744692394970611,100.0,10.0,configuration,2024-11-18 06:12:11
6960,data/TSP/CEPS_generated/88.tsp,1645744692394970611,100.0,10.0,configuration,2024-11-18 06:12:12
6961,data/TSP/CEPS_generated/89.tsp,1645744692394970611,100.0,10.0,configuration,2024-11-18 06:12:12


In [10]:
_ = df.loc[df["comment"] == "test"].pivot_table(index="instance_id", columns="solver_id", values="cost")
_["best"] = np.argmin(_.values, axis=1)
_["min"] = _.iloc[:, :-1].min(axis=1)
_

solver_id,1317975573233644005,1482672950880257277,2300580964560483949,721382164073135269,best,min
instance_id,,,,,,
data/TSP/CEPS_benchmark/cluster/00.tsp,100.0,100.00,100.0,100.00,0,100.00
data/TSP/CEPS_benchmark/cluster/01.tsp,100.0,4.44,100.0,100.00,1,4.44
data/TSP/CEPS_benchmark/cluster/02.tsp,100.0,6.28,100.0,100.00,1,6.28
data/TSP/CEPS_benchmark/cluster/03.tsp,100.0,5.82,100.0,4.81,3,4.81
data/TSP/CEPS_benchmark/cluster/04.tsp,100.0,100.00,100.0,100.00,0,100.00
...,...,...,...,...,...,...
data/TSP/CEPS_benchmark/uniform_portgen/45.tsp,100.0,1.63,100.0,3.66,1,1.63
data/TSP/CEPS_benchmark/uniform_portgen/46.tsp,100.0,1.75,100.0,5.26,1,1.75
data/TSP/CEPS_benchmark/uniform_portgen/47.tsp,100.0,3.82,100.0,1.36,3,1.36


In [11]:
_["min"].describe()

count    470.000000
mean      21.550404
std       37.491389
min        0.210000
25%        2.042500
50%        4.150000
75%        7.750000
max      100.000000
Name: min, dtype: float64

In [12]:
(_["min"] > 90).value_counts()

False    383
True      87
Name: min, dtype: int64

In [ ]:
_.loc[lambda x: x["min"] < 90, "best"].value_counts()

In [ ]:
_.sort_values(by="min")

In [ ]:
# config = df_solvers.loc[df_solvers["id"] == "OJQGNYe__-omEPLuUJkSzOuFdCnYPLzZd99VQqxW2gM="].drop(columns=["id"]).iloc[0].to_dict()
# config = Configuration(configuration_space=CONFIGURATION_SPACE, values=config)
# solver = TSP_LKH_Solver(config=config)

# instance = TSP_Instance(filepath="data/TSP/CEPS_benchmark/cluster_netgen/18.tsp", optimum=10555222.0)

# solver.solve(instance)

In [ ]:
# import json
# with open(DATA_DIR / "TSP" / "index.json") as f:
#     index = json.load(f)

# new_index = {}

# for k, v in index.items():
#     path = k.split("/")
#     file = path[-1]
#     file_no_ext = file.split(".")[0]
#     if "_" in file_no_ext:
#         a, b = file_no_ext.split("_")
#         n = str((int(a) - 4) * 10 + int(b)).zfill(2)
#         new_file = f"{n}.tsp"
#     else:
#         file_no_ext = str(int(file_no_ext)).zfill(2)
#         new_file = f"{file_no_ext}.tsp"
#     new_path = path[:-1] + [new_file]
#     new_k = "/".join(new_path)
#     new_index[new_k] = float(v)

#     path1 = DATA_DIR / k;
#     path2 = DATA_DIR / new_k;
#     path1.rename(path2)

# new_index = {k: new_index[k] for k in sorted(new_index.keys())}

# with open(DATA_DIR / "TSP" / "index.json", "w") as f:
#     json.dump(new_index, f, indent=4)

# new_index